### インポート

In [37]:
 #-*- coding: utf-8 -*-
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
from gensim import corpora, models, matutils
from sklearn.cluster import KMeans
from pprint import pprint
import pickle

### 事前付与ジャンルのリストを作成

In [2]:
pre_genre = ["csAI", "csCC", "csCG", "csCL", "csCV", "csDS", "csGT", "csMA", "csSD", "statAP", "statCO", "statME", "statML", "statTH"]

### 加工CSV読み込み

In [3]:
df = pd.DataFrame()

for genre in pre_genre:
    #df = pd.concat([df, pd.read_csv("./csv/" +  genre + ".csv", header=None)])
    df = pd.concat([df, pd.read_csv("./all_csv/" +  genre + ".txt", delimiter = "|", header=None)])

### 生CSV読み込み

In [5]:
'''
df_raw = pd.DataFrame()

for genre in pre_genre:
    df_raw = pd.concat([df_raw, pd.read_csv("./raw_csv/test_" +  genre + ".csv", header=None)])

df_raw.to_csv("raw_csv_all.csv")
df_raw
'''

'\ndf_raw = pd.DataFrame()\n\nfor genre in pre_genre:\n    df_raw = pd.concat([df_raw, pd.read_csv("./raw_csv/test_" +  genre + ".csv", header=None)])\n\ndf_raw.to_csv("raw_csv_all.csv")\ndf_raw\n'

### データフレームにカラム名をつける

In [6]:
header_names =  ["プログラム実行日時", "論文更新日時", "論文リンク", "PDFリンク", "元論文タイトル", "論文タイトル", "元サマリ", "サマリ", "著者", "事前付与ジャンル"]

In [7]:
#カラム名定義
df.columns = header_names 
#df_raw.columns = header_names[:6]

In [8]:
data_len = len(df)

### カテゴリおよびそれに対応する辞書

In [9]:
cat_names = ["ニューラルネットワーク"
                     , "自然言語処理"
                     , "マーケティング"
                     , "画像解析"
                     , "音声解析"
                     , "強化学習"]

In [10]:
cat_kw = [
  ['neural'
    , 'rnn'
    , 'cnn'
    , 'deep'
    ],
  ['languag'
    , 'word'
    , 'document'
    , 'nlp'],
  ['market'
    , 'custom'
    , 'commerc'
    , 'advertis'
    ],
  ['imag'
    , 'vision'
    , 'photo'
    ],
  ['sound'
  , 'speech'],
  ['reinforc']
]

### 辞書ベースでの教師データ作成、カテゴリ付与

In [11]:
easy_cat = [] #簡易的なカテゴリ振り分けリスト

for n, row in df.iterrows():
    row_cat = []
    for i, cat in enumerate(cat_kw):
        flag = False
        for kw in cat:
            if kw in (row["論文タイトル"] + row["サマリ"]):
                flag = True
        if flag:
            row_cat.append(1)
        else:
            row_cat.append(0)
    easy_cat.append(row_cat)

df_cat = pd.DataFrame(easy_cat)

In [13]:
df.index=range(data_len) #インデックスを設定
#df_raw.index=range(data_len) #生CSVの方にも設定

df_all = df.join(df_cat)
df_all.columns = header_names + cat_names
df_all.head(50)

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,元論文タイトル,論文タイトル,元サマリ,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
0,2017-03-28T14:05:16Z,2017-03-27T16:48:03Z,http://arxiv.org/abs/1703.09179v1,http://arxiv.org/pdf/1703.09179v1,Transfer learning for music classification and...,transfer learn music classif regress task,"In this paper, we present a transfer learning ...",paper present transfer learn approach music cl...,"['Keunwoo Choi', 'György Fazekas', 'Mark Sandl...","['cs.CV', 'cs.AI', 'cs.MM', 'cs.SD']",0,0,0,0,0,0
1,2017-03-28T14:05:16Z,2017-03-27T06:19:38Z,http://arxiv.org/abs/1703.08944v1,http://arxiv.org/abs/1703.08944v1,Intelligent bidirectional rapidly-exploring ra...,intellig bidirect rapid explor random tree opt...,The sampling based motion planning algorithm k...,sampl base motion plan algorithm known rapid e...,"['Ahmed Hussain Qureshi', 'Yasar Ayaz']","['cs.RO', 'cs.AI']",0,0,0,0,0,0
2,2017-03-28T14:05:16Z,2017-03-27T04:03:56Z,http://arxiv.org/abs/1703.08922v1,http://arxiv.org/pdf/1703.08922v1,On Automating the Doctrine of Double Effect,autom doctrin doubl effect,The doctrine of double effect ($\mathcal{DDE}$...,doctrin doubl effect mathcal dde long studi et...,"['Naveen Sundar Govindarajulu', 'Selmer Brings...","['cs.AI', 'cs.LO', 'cs.RO']",1,0,0,0,0,0
3,2017-03-28T14:05:16Z,2017-03-26T19:39:50Z,http://arxiv.org/abs/1703.08862v1,http://arxiv.org/pdf/1703.08862v1,Socially Aware Motion Planning with Deep Reinf...,social awar motion plan deep reinforc learn,For robotic vehicles to navigate safely and ef...,robot vehicl navig safe effici pedestrian rich...,"['Yu Fan Chen', 'Michael Everett', 'Miao Liu',...","['cs.RO', 'cs.AI', 'cs.HC']",1,0,0,0,0,1
4,2017-03-28T14:05:16Z,2017-03-26T16:20:36Z,http://arxiv.org/abs/1703.08840v1,http://arxiv.org/pdf/1703.08840v1,Inferring The Latent Structure of Human Decisi...,infer latent structur human decis make raw vis...,The goal of imitation learning is to match exa...,goal imit learn match exampl expert behavior w...,"['Yunzhu Li', 'Jiaming Song', 'Stefano Ermon']","['cs.LG', 'cs.AI', 'cs.CV']",0,0,0,0,0,1
5,2017-03-28T14:05:16Z,2017-03-26T15:26:34Z,http://arxiv.org/abs/1703.08825v1,http://arxiv.org/pdf/1703.08825v1,Surrogate Model of Multi-Period Flexibility fr...,surrog model multi period flexibl home energi ...,Near-future electric distribution grids operat...,near futur electr distribut grid oper reli dem...,"['Rui Pinto', 'Ricardo Bessa', 'Manuel Matos']","['cs.NE', 'cs.AI']",0,0,1,1,0,0
6,2017-03-28T14:05:16Z,2017-03-26T05:44:56Z,http://arxiv.org/abs/1703.08769v1,http://arxiv.org/pdf/1703.08769v1,Open Vocabulary Scene Parsing,open vocabulari scene pars,Recognizing arbitrary objects in the wild has ...,recogn arbitrari object wild challeng problem ...,"['Hang Zhao', 'Xavier Puig', 'Bolei Zhou', 'Sa...","['cs.CV', 'cs.AI']",0,1,0,1,0,0
7,2017-03-28T14:05:16Z,2017-03-26T03:47:54Z,http://arxiv.org/abs/1703.08762v1,http://arxiv.org/pdf/1703.08762v1,Team Formation for Scheduling Educational Mate...,team format schedul educ materi massiv onlin c...,Whether teaching in a classroom or a Massive O...,whether teach classroom massiv onlin open cour...,"['Sanaz Bahargam', 'Dóra Erdos', 'Azer Bestavr...",['cs.AI'],0,0,0,0,0,0
8,2017-03-28T14:05:16Z,2017-03-25T15:37:09Z,http://arxiv.org/abs/1703.08705v1,http://arxiv.org/pdf/1703.08705v1,Comparing Rule-Based and Deep Learning Models ...,compar rule base deep learn model patient phen...,Objective: We investigate whether deep learnin...,object investig whether deep learn techniqu na...,"['Sebastian Gehrmann', 'Franck Dernoncourt', '...","['cs.CL', 'cs.AI', 'cs.NE', 'stat.ML']",1,1,0,0,0,0
9,2017-03-28T14:05:16Z,2017-03-24T15:43:39Z,http://arxiv.org/abs/1703.08475v1,http://arxiv.org/pdf/1703.08475v1,Overcoming Catastrophic Forgetting by Incremen...,overcom catastroph forget increment moment match,Catastrophic forgetting is a problem which ref...,catastroph forget problem refer lose inform fi...,"['Sang-Woo Lee', 'Jin-Hwa Kim', 'Jung-Woo Ha',...","

In [ ]:
id = 90
test_doc = df_all.loc[id]['論文タイトル']  + df_all.loc[id]['サマリ'] 
df_all.loc[id]

In [ ]:
#df_all.to_csv("df_all.csv")

In [ ]:
documents = df["論文タイトル"] 
#documents = df["サマリ"] 

In [ ]:
texts = [[word for word in document.lower().split()]for document in documents] #単語ごとに抽出

### 辞書作成

In [ ]:
dictionary = corpora.Dictionary(texts)

### 辞書のID数を確認

In [ ]:
id_num = len(dictionary.token2id)

### Bag of words のコーパスを作成

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
tfidf = models.TfidfModel(corpus)

In [ ]:
print (tfidf)

In [ ]:
corpus_tfidf = tfidf[corpus]

### ベクトル化

In [ ]:
vecs = matutils.corpus2dense(corpus_tfidf, num_terms=id_num) #スパースなリスト形式を密にしてsklearnに適用可能な形に

### LSI

In [ ]:
num_topics = 20
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics) # initialize an LSI transformation
pprint(lsi.print_topics(20))
corpus_lsi = lsi[corpus_tfidf]

vecs = []
for i in range(data_len):
    vecs.append(list(matutils.corpus2dense(corpus_lsi, num_terms=num_topics).T[i]))

In [ ]:
# vecs に特徴ベクトルが入ってる！

### k-means クラスタリング

In [ ]:
clusters = KMeans(n_clusters=6, random_state=0).fit_predict(vecs)

In [ ]:
cls_count_list = [0, 0, 0, 0, 0, 0]
cls_list = []

for doc, cls in zip(documents, clusters):
    cls_list.append(cls)
    cls_count_list[cls] += 1

print (cls_count_list)

In [ ]:
df_all['kmeans_predict'] = cls_list

In [ ]:
#df_all.to_csv("df_kmeans_result.csv")

### 仮カテゴリごとのデータフレーム作成

In [14]:
# 学習データのセット
#df_train = df_all[1::2] #偶数 #前半を学習データとする場合
#df_train = df_all #すべてを学習データとする場

df_train =df_all[0:1] #0行目がとれないので
for i in range(2, 11):
    df_train = df_train.append(df_all[i::10])
df_train

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,元論文タイトル,論文タイトル,元サマリ,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
0,2017-03-28T14:05:16Z,2017-03-27T16:48:03Z,http://arxiv.org/abs/1703.09179v1,http://arxiv.org/pdf/1703.09179v1,Transfer learning for music classification and...,transfer learn music classif regress task,"In this paper, we present a transfer learning ...",paper present transfer learn approach music cl...,"['Keunwoo Choi', 'György Fazekas', 'Mark Sandl...","['cs.CV', 'cs.AI', 'cs.MM', 'cs.SD']",0,0,0,0,0,0
2,2017-03-28T14:05:16Z,2017-03-27T04:03:56Z,http://arxiv.org/abs/1703.08922v1,http://arxiv.org/pdf/1703.08922v1,On Automating the Doctrine of Double Effect,autom doctrin doubl effect,The doctrine of double effect ($\mathcal{DDE}$...,doctrin doubl effect mathcal dde long studi et...,"['Naveen Sundar Govindarajulu', 'Selmer Brings...","['cs.AI', 'cs.LO', 'cs.RO']",1,0,0,0,0,0
12,2017-03-28T14:05:20Z,2017-03-24T12:07:34Z,http://arxiv.org/abs/1703.08383v1,http://arxiv.org/pdf/1703.08383v1,Smart Augmentation - Learning an Optimal Data ...,smart augment learn optim data augment strategi,A recurring problem faced when training neural...,recur problem face train neural network typic ...,"['Joseph Lemley', 'Shabab Bazrafkan', 'Peter C...","['cs.AI', 'cs.LG', 'stat.ML']",1,0,0,0,0,0
22,2017-03-28T14:05:24Z,2017-03-22T19:08:48Z,http://arxiv.org/abs/1703.07822v1,http://arxiv.org/pdf/1703.07822v1,Information-theoretic Model Identification and...,inform theoret model identif polici search use...,We consider the problem of a robot learning th...,consid problem robot learn mechan properti obj...,"['Shaojun Zhu', 'Andrew Kimmel', 'Abdeslam Bou...","['cs.RO', 'cs.AI', 'cs.LG']",0,0,0,0,0,1
32,2017-03-28T14:05:28Z,2017-03-22T00:24:03Z,http://arxiv.org/abs/1703.07326v2,http://arxiv.org/pdf/1703.07326v2,One-Shot Imitation Learning,one shot imit learn,Imitation learning has been commonly applied t...,imit learn common appli solv differ task isol ...,"['Yan Duan', 'Marcin Andrychowicz', 'Bradly C....","['cs.AI', 'cs.LG', 'cs.NE', 'cs.RO']",1,0,0,0,0,0
42,2017-03-28T14:05:32Z,2017-03-20T09:28:38Z,http://arxiv.org/abs/1703.06642v1,http://arxiv.org/pdf/1703.06642v1,Towards a Quantum World Wide Web,toward quantum world wide web,We elaborate a quantum model for corpora of wr...,elabor quantum model corpora written document ...,"['Diederik Aerts', 'Jonito Aerts Arguelles', '...","['cs.AI', 'cs.CL', 'quant-ph']",0,1,0,0,0,0
52,2017-03-28T14:05:37Z,2017-03-21T14:26:33Z,http://arxiv.org/abs/1703.06207v2,http://arxiv.org/pdf/1703.06207v2,Cooperating with Machines,cooper machin,Since Alan Turing envisioned Artificial Intell...,sinc alan ture envis artifici intellig ai majo...,"['Jacob W. Crandall', 'Mayada Oudah', 'Tennom'...",['cs.AI'],0,0,0,0,0,0
62,2017-03-28T14:05:41Z,2017-03-16T01:31:33Z,http://arxiv.org/abs/1703.05449v1,http://arxiv.org/pdf/1703.05449v1,Minimax Regret Bounds for Reinforcement Learning,minimax regret bound reinforc learn,We consider the problem of efficient explorati...,consid problem effici explor finit horizon mdp...,"['Mohammad Gheshlaghi Azar', 'Ian Osband', 'Ré...","['stat.ML', 'cs.AI', 'cs.LG']",0,0,0,0,0,1
72,2017-03-28T14:05:45Z,2017-03-17T00:56:18Z,http://arxiv.org/abs/1703.04912v2,http://arxiv.org/pdf/1703.04912v2,Syntax-Preserving Belief Change Operators for ...,syntax preserv belief chang oper logic program,Recent methods have adapted the well-establish...,recent method adapt well establish agm belief ...,"['Sebastian Binnewies', 'Zhiqiang Zhuang', 'Ke...","['cs.AI', 'I.2.3; I.2.4; F.4.1']",0,0,0,0,0,0
82,2017-03-28T14:05:50Z,2017-03-13T17:34:18Z,http://arxiv.org/abs/1703.04498v1,http://arxiv.org/pdf/1703.04498v1,High-Throughput and Language-Agnostic Entity D...,high throughput languag agnost entiti disambig...,The Entity Disambiguation and Linking (EDL) ta...,entiti disambigu link edl task match entiti me...,"['Preeti Bhargava', 'Nemanja Spasojevic', 'Guo...","['cs.IR', 'cs.AI', 'cs.CL']",0,1,0,0,0,0


In [15]:
# カテゴリごとのデータフレームの表示（検証用）
df_img = df_all[df_all["画像解析"] == 1]
for i, row in df_img.iterrows():
    print ("タイトル(" + str(len(row["論文タイトル"].split())) +"):" + row["論文タイトル"])
    print ("サマリ(" + str(len(row["サマリ"].split())) +"):" + row["サマリ"] + "\n")

タイトル(9):surrog model multi period flexibl home energi manag system
サマリ(132):near futur electr distribut grid oper reli demand side flexibl implement demand respons strategi take advantag intellig manag increas common small scale energi storag home energi manag system hem play crucial role flexibl provis system oper market player like aggreg model multi period flexibl residenti consum hem flexibl batteri storag electr water heater compli intern constraint comfort level data privaci uncertainti complex task paper describ comput method capabl effici defin learn feasibl flexibl set control resourc connect hem evolutionari particl swarm optim epso algorithm adopt reshap deriv set feasibl tempor trajectori residenti net load consid storag flexibl applianc predefin costum prefer well load photovolta pv forecast uncertainti support vector data descript svdd algorithm use build model capabl classifi feasibl unfeas hem oper trajectori upon request optim control algorithm oper dso market player



In [16]:
df_cat_list = [] #各ジャンルのタグがついたデータフレームを保持するリスト

for cat in cat_names:
    tmp = df_train[df_train[cat] == 1]
    df_cat_list.append(tmp)

## ナイーブベイズ分類

### $P(cat)$ の算出

In [17]:
pcat_list = []

for df_cat in df_cat_list:
    pcat_list.append(len(df_cat))
    
pcat_list = np.array(pcat_list)/(sum(pcat_list) * 1.0)
pcat_list #P(cat)

array([ 0.3289689 ,  0.19967267,  0.06219313,  0.21767594,  0.12602291,
        0.06546645])

### $P(word_i | cat) $ の算出

In [38]:
title_weight = 1 #default # タイトルに乗算する重み

word_dict_list = []
word_num_all_list = []

for df_cat in df_cat_list:
    word_list = []
    word_dict = {}
    for i, row in df_cat.iterrows():
        title_weight = 15
        #title_weight = round(len(row["サマリ"].split()) / len(row["論文タイトル"].split()))
        #word_list += (row["論文タイトル"] + row["サマリ"]).split(" ")
        word_list += row["論文タイトル"].split() * title_weight
        
    word_num_all = len(word_list) #カテゴリ内の総単語数
    word_num_all_list.append(word_num_all)
    
    for word in word_list:
        if word in word_dict:
            word_dict[word] += 1
        else:
            word_dict[word] = 1 #+ 1# スムージング用に１を足しておく
    
    for word in word_dict:
        word_dict[word] = 1.0 * word_dict[word] / word_num_all
    
    word_dict_list.append(word_dict)

### ピックル (pickle) でリスト、辞書をファイルに保存

In [44]:
with open('pcat_list.dump', 'wb') as f:
    pickle.dump(pcat_list, f)

with open('word_dict_list.dump', 'wb') as f:
    pickle.dump(word_dict_list, f)
    
with open('word_num_all_list.dump', 'wb') as f:
    pickle.dump(word_num_all_list, f)

In [19]:
word_dict_list[3]['imag']

0.03487423521414004

### デモ用

In [20]:
document = '''
imag imag imag imag imag imag cosin normal use cosin similar instead dot product neural networktradit multi layer neural network use dot product output vector previous layer incom weight vector input activ function result dot product unbound thus increas risk larg varianc larg varianc neuron make model sensit chang input distribut thus result poor general aggrav intern covari shift slow train bound dot product decreas varianc propos use cosin similar instead dot product neural network call cosin normal experi show cosin normal fulli connect neural network notabl reduc test err lower diverg compar normal techniqu appli convolut network cosin normal also signific enhanc accuraci classif acceler train
'''

#document = test_doc
print(document)
words = document.split()

cat_prob = []
for i, word_dict in enumerate(word_dict_list):
    prob = np.log(pcat_list[i]) #初期化
    for word in words:
        if word in word_dict:
            prob += np.log(word_dict[word])
        else:
            prob += np.log(1.0/word_num_all_list[i])
    cat_prob.append(prob)

print(cat_prob)
print((cat_prob - np.mean(cat_prob))/np.std(cat_prob))
print(np.argmax(np.array(cat_prob)))


imag imag imag imag imag imag cosin normal use cosin similar instead dot product neural networktradit multi layer neural network use dot product output vector previous layer incom weight vector input activ function result dot product unbound thus increas risk larg varianc larg varianc neuron make model sensit chang input distribut thus result poor general aggrav intern covari shift slow train bound dot product decreas varianc propos use cosin similar instead dot product neural network call cosin normal experi show cosin normal fulli connect neural network notabl reduc test err lower diverg compar normal techniqu appli convolut network cosin normal also signific enhanc accuraci classif acceler train

[-837.59373712462332, -866.64215057591366, -847.24290312984999, -841.16275694564695, -834.18873790722671, -837.95026252467937]
[ 0.60298531 -2.07676029 -0.28716004  0.27373961  0.91709994  0.57009548]
4


### 精度評価

In [21]:
 # テストデータのセット
#df_test = df_all[0::2]
df_test =df_all[1::10]
df_test

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,元論文タイトル,論文タイトル,元サマリ,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
1,2017-03-28T14:05:16Z,2017-03-27T06:19:38Z,http://arxiv.org/abs/1703.08944v1,http://arxiv.org/abs/1703.08944v1,Intelligent bidirectional rapidly-exploring ra...,intellig bidirect rapid explor random tree opt...,The sampling based motion planning algorithm k...,sampl base motion plan algorithm known rapid e...,"['Ahmed Hussain Qureshi', 'Yasar Ayaz']","['cs.RO', 'cs.AI']",0,0,0,0,0,0
11,2017-03-28T14:05:20Z,2017-03-24T13:00:52Z,http://arxiv.org/abs/1703.08397v1,http://arxiv.org/pdf/1703.08397v1,Reasoning by Cases in Structured Argumentation,reason case structur argument,We extend the $ASPIC^+$ framework for structur...,extend aspic framework structur argument allow...,"['Mathieu Beirlaen', 'Jesse Heyninck', 'Christ...","['cs.AI', '68T27', 'I.2.3; I.2.4']",0,0,0,0,0,0
21,2017-03-28T14:05:24Z,2017-03-23T04:25:48Z,http://arxiv.org/abs/1703.07928v1,http://arxiv.org/pdf/1703.07928v1,Guided Perturbations: Self Corrective Behavior...,guid perturb self correct behavior convolut ne...,Convolutional Neural Networks have been a subj...,convolut neural network subject great import p...,"['Swami Sankaranarayanan', 'Arpit Jain', 'Ser ...","['cs.CV', 'cs.AI', 'stat.ML']",1,0,0,1,0,0
31,2017-03-28T14:05:28Z,2017-03-21T18:29:05Z,http://arxiv.org/abs/1703.07381v1,http://arxiv.org/pdf/1703.07381v1,Improving Statistical Multimedia Information R...,improv statist multimedia inform retriev model...,A typical IR system that delivers and stores i...,typic ir system deliv store inform affect prob...,"['Gagandeep Singh Narula', 'Vishal Jain']","['cs.IR', 'cs.AI']",0,1,0,0,0,0
41,2017-03-28T14:05:32Z,2017-03-20T11:44:00Z,http://arxiv.org/abs/1703.06692v1,http://arxiv.org/pdf/1703.06692v1,QMDP-Net: Deep Learning for Planning under Par...,qmdp net deep learn plan partial observ,"This paper introduces QMDP-net, a neural netwo...",paper introduc qmdp net neural network archite...,"['Peter Karkus', 'David Hsu', 'Wee Sun Lee']","['cs.AI', 'cs.LG', 'cs.NE', 'stat.ML']",1,0,0,0,0,0
51,2017-03-28T14:05:37Z,2017-03-18T09:04:05Z,http://arxiv.org/abs/1703.06275v1,http://arxiv.org/pdf/1703.06275v1,Evolving Game Skill-Depth using General Video ...,evolv game skill depth use general video game ...,"Most games have, or can be generalised to have...",game generalis number paramet may vari order p...,"['Jialin Liu', 'Julian Togelius', 'Diego Perez...",['cs.AI'],0,0,0,0,0,0
61,2017-03-28T14:05:41Z,2017-03-16T01:37:25Z,http://arxiv.org/abs/1703.05452v1,http://arxiv.org/pdf/1703.05452v1,Efficient Online Learning for Optimizing Value...,effici onlin learn optim valu inform theori ap...,We consider the optimal value of information (...,consid optim valu inform voi problem goal sequ...,"['Yuxin Chen', 'Jean-Michel Renders', 'Morteza...","['cs.AI', 'cs.LG', 'stat.ML']",0,0,0,0,0,0
71,2017-03-28T14:05:45Z,2017-03-15T05:43:48Z,http://arxiv.org/abs/1703.04940v1,http://arxiv.org/pdf/1703.04940v1,Resilience: A Criterion for Learning in the Pr...,resili criterion learn presenc arbitrari outlier,"We introduce a criterion, resilience, which al...",introduc criterion resili allow properti datas...,"['Jacob Steinhardt', 'Moses Charikar', 'Gregor...","['cs.LG', 'cs.AI', 'cs.CC', 'cs.CR', 'stat.ML']",0,0,0,0,0,0
81,2017-03-28T14:05:50Z,2017-03-13T17:58:36Z,http://arxiv.org/abs/1703.04529v1,http://arxiv.org/pdf/1703.04529v1,Task-based End-to-end Model Learning,task base end end model learn,As machine learning techniques have become mor...,machin learn techniqu becom ubiquit becom comm...,"['Priya L. Donti', 'Brandon Amos', 'J. Zico Ko...","['cs.LG', 'cs.AI']",0,0,0,0,0,0
91,2017-03-28T14:05:54Z,2017-03-15T08:16:37Z,http://arxiv.org/abs/1703.04159v2,http://arxiv.org/pdf/1703.04159v2,Any-Angle Pathfinding for Multiple Agents Base...,ani angl pathfind multipl agent base sipp algo...,The problem of finding conflict-free trajector...,problem find conflict free trajectori multipl ...,"['Ko

### タイトルに重みをつける
重みをつけないと、タイトルに重要な情報があるにも関わらず、サマリのどうでもいい単語に引っ張られて分類精度が下がる。フェアにするために、(サマリに含まれる単語数/タイトルに含まれる単語数)だけディクショナリに登録するときの数値に重みをつけておく。

In [22]:
title_weight = 1 #default # タイトルに乗算する重み

In [34]:
correct = 0 #正解数
wrong = 0
uncover = 0 # タグが付与されていない論文の数
data_len = len(df_test)
for j, row in df_test.iterrows(): #元のデータフレームからテキストを抽出
    title_weight = round(len(row["サマリ"].split()) / len(row["論文タイトル"].split()))

    document = (row["論文タイトル"] + " ") * title_weight + row["サマリ"]
    words = document.split()

    cat_prob = []
    for i, word_dict in enumerate(word_dict_list):
        prob = np.log(pcat_list[i]) #初期化
        for word in words:
            if word in word_dict:
                prob += np.log(word_dict[word])
            else:
                prob += np.log(1.0/word_num_all_list[i])
        cat_prob.append(prob)

    flag = False #デフォルト
    for cat in cat_names:
        if row[cat] == 1:
            flag = True #評価対象

    if flag == True: #評価対象のものに限定して
        if row[cat_names[np.argmax(np.array(cat_prob))]] == 1:  #予測ラベルの列が1かどうか
            correct += 1
        else:
            wrong += 1
    else:
        uncover += 1
    print(j)

print ("正解率: " + str(1.0 * correct / (data_len - uncover)))
print ("カバー率: " + str(1.0 * (data_len - uncover) / data_len))
print(cat_prob)
print((cat_prob - np.mean(cat_prob))/np.std(cat_prob))
document

9
1
17
11
12
21
8
31
12
41
13
51
12
61
7
71
14
81
13
91
20
101
10
111
8
121
14
131
15
141
6
151
11
161
45
171
13
181
30
191
21
201
22
211
17
221
27
231
10
241
14
251
10
261
10
271
11
281
20
291
13
301
16
311
7
321
19
331
14
341
12
351
10
361
12
371
8
381
12
391
17
401
26
411
17
421
16
431
10
441
9
451
13
461
12
471
23
481
15
491
18
501
20
511
17
521
19
531
9
541
20
551
11
561
8
571
17
581
12
591
35
601
3
611
15
621
40
631
9
641
21
651
36
661
14
671
20
681
22
691
14
701
12
711
9
721
17
731
13
741
31
751
19
761
18
771
18
781
20
791
23
801
22
811
17
821
6
831
10
841
20
851
16
861
26
871
13
881
23
891
20
901
12
911
16
921
15
931
16
941
11
951
21
961
7
971
20
981
15
991
23
1001
25
1011
11
1021
15
1031
12
1041
13
1051
11
1061
20
1071
7
1081
6
1091
9
1101
15
1111
19
1121
6
1131
22
1141
9
1151
9
1161
15
1171
22
1181
18
1191
14
1201
10
1211
8
1221
14
1231
11
1241
13
1251
13
1261
12
1271
62
1281
12
1291
13
1301
2
1311
10
1321
6
1331
34
1341
6
1351
8
1361
8
1371
14
1381
7
1391
正解率: 0.523809523809

### マルチタグ付与の閾値を分類によって求める

In [24]:
pred_list = []

# 分類用データ
label_list = [] # ラベル
data_list = [] # 教師データ

for k, row in df_all.iterrows(): #元のデータフレームからテキストを抽出
    document = row["論文タイトル"] + row["サマリ"]
    words = document.split()

    cat_prob = []
    for i, word_dict in enumerate(word_dict_list):
        prob = np.log(pcat_list[i]) #初期化
        for word in words:
            if word in word_dict:
                prob += np.log(word_dict[word])
            else:
                prob += np.log(1.0/word_num_all_list[i])
        cat_prob.append(prob)
    
    # ラベル作成
    for cat in cat_names: 
        if row[cat] == 1:
            label_list.append(1)
        else:
            label_list.append(0)
        
    # 教師データ作成
    data_list += list((cat_prob - np.mean(cat_prob))/np.std(cat_prob))

x = np.array(label_list)
y = np.array(data_list)

# ２つの正規分布の確率密度が一致する点を求める
p0 = len(y[x==0])/len(y)
m0 = np.mean(y[x==0])
s0 = np.std(y[x==0])
p1 = len(y[x==1])/len(y)
m1 = np.mean(y[x==1])
s1 = np.std(y[x==1])

# ２つの確率密度関数の交点を求める　
thrs = (-(m0 * s1**2 - m1 * s0**2) \
     - s1 * s0 * np.sqrt((s1 - s0)**2 + (s0**2 - s1**2) * np.log((s1*2*p0**2)/s0**2**p1**2)) \
    / (s0**2 - s1**2))

print(x)
print(y)

print("閾値: ")
print(thrs)

thr_wrong = (sum(abs(x - (y>thrs))))
print ("マルチタグ付与精度：")
print (1.0 * (len(y) - thr_wrong) / len(y))

[0 0 0 ..., 0 0 0]
[ 1.30678964 -0.96150132 -1.2588737  ..., -1.01347451  0.21016079
  1.71880994]
閾値: 
-0.357071605274
マルチタグ付与精度：
0.494642857143


### 実運用用のプログラム

In [25]:
# データ受け取り: df_recv
df_recv = df #dfで仮置き
threshold = 1.2 #可変

pred_list = []
pred_result = []
for j, row in df_recv.iterrows(): #元のデータフレームからテキストを抽出
    document = row["論文タイトル"] + row["サマリ"]
    words = document.split()

    cat_prob = []
    for i, word_dict in enumerate(word_dict_list):
        prob = np.log(pcat_list[i]) #初期化
        for word in words:
            if word in word_dict:
                prob += np.log(word_dict[word])
            else:
                prob += np.log(1.0/word_num_all_list[i])
        cat_prob.append(prob)
    pred_result.append(list((cat_prob - np.mean(cat_prob))/np.std(cat_prob) > threshold)) #事前に算出した閾値
    
    df_pred = pd.DataFrame(pred_result)
    df_all = df.join(df_pred)
    
df_all.columns = header_names + cat_names
df_all

,プログラム実行日時,論文更新日時,論文リンク,PDFリンク,元論文タイトル,論文タイトル,元サマリ,サマリ,著者,事前付与ジャンル,ニューラルネットワーク,自然言語処理,マーケティング,画像解析,音声解析,強化学習
0,2017-03-28T14:05:16Z,2017-03-27T16:48:03Z,http://arxiv.org/abs/1703.09179v1,http://arxiv.org/pdf/1703.09179v1,Transfer learning for music classification and...,transfer learn music classif regress task,"In this paper, we present a transfer learning ...",paper present transfer learn approach music cl...,"['Keunwoo Choi', 'György Fazekas', 'Mark Sandl...","['cs.CV', 'cs.AI', 'cs.MM', 'cs.SD']",True,False,False,False,False,False
1,2017-03-28T14:05:16Z,2017-03-27T06:19:38Z,http://arxiv.org/abs/1703.08944v1,http://arxiv.org/abs/1703.08944v1,Intelligent bidirectional rapidly-exploring ra...,intellig bidirect rapid explor random tree opt...,The sampling based motion planning algorithm k...,sampl base motion plan algorithm known rapid e...,"['Ahmed Hussain Qureshi', 'Yasar Ayaz']","['cs.RO', 'cs.AI']",False,False,False,False,False,True
2,2017-03-28T14:05:16Z,2017-03-27T04:03:56Z,http://arxiv.org/abs/1703.08922v1,http://arxiv.org/pdf/1703.08922v1,On Automating the Doctrine of Double Effect,autom doctrin doubl effect,The doctrine of double effect ($\mathcal{DDE}$...,doctrin doubl effect mathcal dde long studi et...,"['Naveen Sundar Govindarajulu', 'Selmer Brings...","['cs.AI', 'cs.LO', 'cs.RO']",False,False,True,False,False,True
3,2017-03-28T14:05:16Z,2017-03-26T19:39:50Z,http://arxiv.org/abs/1703.08862v1,http://arxiv.org/pdf/1703.08862v1,Socially Aware Motion Planning with Deep Reinf...,social awar motion plan deep reinforc learn,For robotic vehicles to navigate safely and ef...,robot vehicl navig safe effici pedestrian rich...,"['Yu Fan Chen', 'Michael Everett', 'Miao Liu',...","['cs.RO', 'cs.AI', 'cs.HC']",False,False,False,False,False,True
4,2017-03-28T14:05:16Z,2017-03-26T16:20:36Z,http://arxiv.org/abs/1703.08840v1,http://arxiv.org/pdf/1703.08840v1,Inferring The Latent Structure of Human Decisi...,infer latent structur human decis make raw vis...,The goal of imitation learning is to match exa...,goal imit learn match exampl expert behavior w...,"['Yunzhu Li', 'Jiaming Song', 'Stefano Ermon']","['cs.LG', 'cs.AI', 'cs.CV']",False,False,False,False,False,True
5,2017-03-28T14:05:16Z,2017-03-26T15:26:34Z,http://arxiv.org/abs/1703.08825v1,http://arxiv.org/pdf/1703.08825v1,Surrogate Model of Multi-Period Flexibility fr...,surrog model multi period flexibl home energi ...,Near-future electric distribution grids operat...,near futur electr distribut grid oper reli dem...,"['Rui Pinto', 'Ricardo Bessa', 'Manuel Matos']","['cs.NE', 'cs.AI']",False,False,True,False,False,False
6,2017-03-28T14:05:16Z,2017-03-26T05:44:56Z,http://arxiv.org/abs/1703.08769v1,http://arxiv.org/pdf/1703.08769v1,Open Vocabulary Scene Parsing,open vocabulari scene pars,Recognizing arbitrary objects in the wild has ...,recogn arbitrari object wild challeng problem ...,"['Hang Zhao', 'Xavier Puig', 'Bolei Zhou', 'Sa...","['cs.CV', 'cs.AI']",False,True,False,False,False,False
7,2017-03-28T14:05:16Z,2017-03-26T03:47:54Z,http://arxiv.org/abs/1703.08762v1,http://arxiv.org/pdf/1703.08762v1,Team Formation for Scheduling Educational Mate...,team format schedul educ materi massiv onlin c...,Whether teaching in a classroom or a Massive O...,whether teach classroom massiv onlin open cour...,"['Sanaz Bahargam', 'Dóra Erdos', 'Azer Bestavr...",['cs.AI'],False,False,True,False,False,True
8,2017-03-28T14:05:16Z,2017-03-25T15:37:09Z,http://arxiv.org/abs/1703.08705v1,http://arxiv.org/pdf/1703.08705v1,Comparing Rule-Based and Deep Learning Models ...,compar rule base deep learn model patient phen...,Objective: We investigate whether deep learnin...,object investig whether deep learn techniqu na...,"['Sebastian Gehrmann', 'Franck Dernoncourt', '...","['cs.CL', 'cs.AI', 'cs.NE', 'stat.ML']",False,True,False,False,False,False
9,2017-03-28T14:05:16Z,2017-03-24T15:43:39Z,http://arxiv.org/abs/1703.08475v1,http://arxiv.org/pdf/1703.08475v1,Overcoming Catastrophic Forgetting by Incremen.

In [27]:
for cat in cat_names:
    df_classified = df_all[df_all[cat]== True]
    #df_classified.to_csv("./sample_csv/" + cat + ".csv")
    for i, row in df_classified.iterrows():
        link = row["論文リンク"]
        link = 'http://arxiv.org/abs/1702.04031v1' #これは消す
        df_hit = df_raw[df_raw["論文リンク"] == link]

        for j, hit_row in df_hit.iterrows():
            print(hit_row["論文タイトル"]) \
            , print(row["著者"].replace("|", ", ")) \
            , print(row["論文リンク"])
            break # 該当行は１つのため

KeyError: '論文リンク'

In [ ]:
#分類後のカテゴリごとの結果表示(検証用)
df_classified = df_all[df_all["ニューラルネットワーク"] == 1]

for i, row in df_classified.iterrows():
    print("タイトル: " + row["元論文タイトル"])
    print("サマリ: " + row["元サマリ"] + "\n")

### k-NN 分類器
あまりうまくいかなかった。単純に閾値を使ったほうがベター。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# k-NN
x = np.array(label_list)
y = np.array(data_list).reshape(-1, 1) # 1次元情報は再構成の必要
    
z = np.array([0.78510301, -0.35518885, -1.67253257, 1.46370845, -0.4934328, 0.27234276]).reshape(-1, 1)

knn = KNeighborsClassifier(n_neighbors=11, weights='distance')
knn.fit(y, x)
output = knn.predict(0.09139588)

## Complement naive bayes classifier

### $P(word_i | \bar{cat}) $ の算出

In [ ]:
word_dict_list = [{}, {}, {}, {}, {}, {}]
word_num_all_list = []

for cat_id, df_cat in enumerate(df_cat_list):
    word_list = []
    for i, row in df_cat.iterrows():
        word_list += (row["論文タイトル"] + row["サマリ"]).split(" ")
        
    word_num_all = len(word_list) #カテゴリ内の総単語数
    word_num_all_list.append(word_num_all)
    
    for word in word_list:
        for list_id, word_dict in enumerate(word_dict_list):
            if list_id != cat_id: #今みているカテゴリ以外のカテゴリに対して
                if word in word_dict:
                    word_dict[word] += 1
                else:
                    word_dict[word] = 1 #ディクショナリに存在しなければ新たに登録

for i, word_dict in enumerate(word_dict_list):
    for word in word_dict:
        word_dict[word] = 1.0 * word_dict[word] / word_num_all_list[i]

###  テスト・評価（Complement naive bayes classifier）

### 精度評価

In [ ]:
pred_list = []
for k, row in df.iterrows():
    document = row["論文タイトル"] + row["サマリ"]
    words = document.split()

    cat_prob = []
    for i, word_dict in enumerate(word_dict_list):
        prob = np.log(pcat_list[i]) #初期化
        for word in words:
            if word in word_dict:
                prob += np.log(1.0 /word_dict[word])
            else:
                prob += np.log(word_num_all_list[i])
        cat_prob.append(prob)

    pred_list.append(np.argmax(np.array(cat_prob)))

uncover = 0 # タグが付与されていない論文の数
correct = 0 #正解数
wrong = 0

for i, row in df_all.iterrows():
        flag = False #デフォルト
        for cat in cat_names:
            if row[cat] == 1:
                flag = True #評価対象

        if flag == True: #評価対象のものに限定して
            if row[cat_names[pred_list[i]]] == 1:  #予測ラベルの列が1かどうか
                correct += 1
        else:
            wrong += 1
    else:
        uncover += 1

print ("正解率: " + str(1.0 * correct / (data_len - uncover)))
print ("カバー率: " + str(1.0 * (data_len - uncover) / data_len))

## Negation naive bayes

### $P(word_i | \bar{cat}) $ の算出

In [ ]:
word_dict_list = [{}, {}, {}, {}, {}, {}]
word_num_all_list = []

for cat_id, df_cat in enumerate(df_cat_list):
    word_list = []
    for i, row in df_cat.iterrows():
        word_list += (row["論文タイトル"] + row["サマリ"]).split(" ")
        
    word_num_all = len(word_list) #カテゴリ内の総単語数
    word_num_all_list.append(word_num_all)
    
    for word in word_list:
        for list_id, word_dict in enumerate(word_dict_list):
            if list_id != cat_id: #今みているカテゴリ以外のカテゴリに対して
                if word in word_dict:
                    word_dict[word] += 1
                else:
                    word_dict[word] = 1 #ディクショナリに存在しなければ新たに登録

for i, word_dict in enumerate(word_dict_list):
    for word in word_dict:
        word_dict[word] = 1.0 * word_dict[word] / word_num_all_list[i]

###  テスト・評価（Negation naive bayes classifier）

### 精度評価

In [ ]:
print(len(word_dict_list))

In [ ]:
pred_list = []
for k, row in df.iterrows():
    document = row["論文タイトル"] + row["サマリ"]
    words = document.split()

    cat_prob = []
    for i, word_dict in enumerate(word_dict_list):
        prob = np.log(1.0 / (1.0 - pcat_list[i])) #初期化
        for word in words:
            if word in word_dict:
                prob += np.log(1.0 /word_dict[word])
            else:
                prob += np.log(word_num_all_list[i])
        cat_prob.append(prob)
    
    pred_list.append(np.argmax(np.array(cat_prob)))

uncover = 0 # タグが付与されていない論文の数
correct = 0 #正解数
wrong = 0

for i, row in df_all.iterrows():
    flag = False #デフォルト
    for cat in cat_names:
        if row[cat] == 1:
            flag = True #評価対象
    
    if flag == True: #評価対象のものに限定して
        if row[cat_names[pred_list[i]]] == 1:  #予測ラベルの列が1かどうか
            correct += 1
        else:
            wrong += 1
    else:
        uncover += 1

print ("正解率: " + str(1.0 * correct / (data_len - uncover)))
print ("カバー率: " + str(1.0 * (data_len - uncover) / data_len))